<a href="https://colab.research.google.com/github/GURJEW/grammemer/blob/main/grammemer_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GRAMMEMER
PyTorch edition for Colab<br>
Version 1.0.2

---
---

## MINING

In [1]:
#@title MOUNTING DRIVE
import os
import sys

import pandas as pd
import torch as pth

from google.colab import drive
from IPython.display import clear_output




#@markdown ***Enter root directory and path to grammemer:***
root = '/content/gdrive/My Drive/Projects/Grammemer/' #@param {type: 'string'}
path = '/content/gdrive/My Drive/Projects/Grammemer/' #@param {type: 'string'}


drive.mount('/content/gdrive')
os.chdir(root)
sys.path.append(path)


from grammemer import Grammemer
print('The Grammemer is imported.')

#@markdown > **root** -- корневая директория<br>
#@markdown > **path** -- директория, в которую установлен пакет grammemer

#@markdown **RUN CELL: to mount gdrive.**



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
The Grammemer is imported.


---

In [2]:
#@title PANDAS OPTIONS
max_columns = None  #@param {type: 'raw'}
multi_sparse = True  #@param ['True', 'False'] {type: 'raw'}
width = 120  #@param {type: 'integer'}
colheader_justify = 'left'  #@param ['left', 'center', 'right']
accuracy = 2  #@param {type: 'integer'}
use_eng_prefix = True  #@param ['True', 'False'] {type: 'raw'}

pd.set_option('display.max_columns', max_columns)
pd.set_option('display.multi_sparse', multi_sparse)
pd.set_option('display.width', width)
pd.set_option('display.colheader_justify', colheader_justify)
pd.set_eng_float_format(accuracy=accuracy, use_eng_prefix=use_eng_prefix)

---

In [3]:
#@title MODEL SETTING
#@markdown ***Enter model and data dimensions:***

depth = 5  #@param {type: 'integer'}
embedding =    128#@param {type: 'integer'}
sentence = 30  #@param {type: 'integer'}
word = 30  #@param {type: 'integer'}

#@markdown > **depth** -- количество свёрточных блоков в модулях обработки уровня предложений и слов <br>
#@markdown > **embedding** -- размерность пространства вложения закодированных символов (33 кириллических буквы и дефис)<br>
#@markdown > **sentence** -- максимальная длина предложений (изменение требует повторной векторизации данных) <br>
#@markdown > **word** -- максимальная длина слов (изменение требует повторной векторизации данных)

#@markdown **RUN CELL: to create grammemer as instance of Grammemer.**


setting = {
    'depth': depth,
    'embedding': embedding,
    'sentence': sentence,
    'word': word
}
grammemer = Grammemer(**setting)
print('The grammemer is created.')


The grammemer is created.




---



In [4]:
#@title TOKENIZE COLLECTION
#@markdown ***Enter paths to the compressed texts and the directory to tokenized documents:***
zip_file = 'data/fb2_demo.zip' #@param {type: 'string'}
doc_dir = 'data/docs_demo' #@param {type: 'string'}

#@markdown > **zip_file** -- путь к архиву с fb2-документами <br>
#@markdown > **doc_dir** -- путь к директории для токенезированных текстов

#@markdown **RUN CELL: to tokenize collection.**

grammemer.data.tokenize(zip_file, doc_dir)

Progress [tokenizer]: ################################################################

[100.0%]֍[0:0:1]

---

In [5]:
#@title TAGGING WITH PYMORPHY2 
doc_dir = 'data/docs_demo'  #@param {type: 'string'}
tag_dir = 'data/tags_demo'  #@param {type: 'string'}
occurrences = 'data/occurrences.csv'  #@param {type:'string'}

#@markdown > **doc_dir** -- путь к директории токенезированных текстов<br>
#@markdown > **tag_dir** -- путь к директории для соответствующих тегов<br>
#@markdown > **occurrences** -- путь к файлу с частотами попарной встречаемости граммем

#@markdown **RUN CELL: to tag based on the probability estimation of a sequence of grammemes.**

!pip install pymorphy2[fast]
clear_output()

grammemer.data.tag(doc_dir, tag_dir, occurrences)

Progress [tagger]: ################################################################

[100.0%]֍[0:0:0]

---
---

## LEARNING

In [6]:
#@title READING
#@markdown ***Enter paths to data:***
path = None  #@param {type: 'raw'}
doc_dir = 'data/docs_demo'  #@param {type: 'string'}
tag_dir = 'data/tags_demo'  #@param {type: 'string'}
suffix = '_demo'  #@param {type: 'string'}

#@markdown > **path** -- путь до tsv-файла (если он был создан)<br>
#@markdown > **doc_dir** -- путь к директории токенезированных текстов<br>
#@markdown > **tag_dir** -- путь к директории с файлами тегов<br>
#@markdown > **suffix** -- суффикс с которым будет сохранён фрейм данных

#@markdown **RUN CELL: to read documents and create DataFrame for vectorizing.**


grammemer.data.read(path=path, doc_dir=doc_dir, tag_dir=tag_dir)
grammemer.data.write(suffix)

Progress [reader]: ################################################################

[100.0%]֍[0:0:2]

---

In [7]:
#@title VECTORIZE DATA

suffix = '_demo'  #@param {type: 'string'}
del_source = False  #@param ['True', 'False'] {type: 'raw'}

#@markdown > **suffix** -- суффикс с которым будyт сохранёны входные (x) и выходные (y) данные<br>
#@markdown > **del_source** -- нужно ли удалять фрейм после векторизации

#@markdown **RUN CELL: to vectorize data and save inputs (x) and targets (y).**


grammemer.data.vectorize(del_source=del_source)
grammemer.data.save(suffix)

Progress [vectorizer]: ################################################################

[100.0%]֍[0:0:6]


Векторизация данных завершена.


---

In [8]:
#@title SPLIT DATASET

suffix = '_demo'  #@param {type: 'string'}
ratio = '8/1/1'  #@param {type: 'string'}
del_source = False  #@param ['True', 'False'] {type: 'raw'}
shuffle = True  #@param ['True', 'False'] {type: 'raw'}

#@markdown > **suffix** -- суффикс с которым будyт сохранёны входные (x) и выходные (y) данные<br>
#@markdown > **ratio** -- отношение частей разделённого датасета (например, "8/2" или "5/2/1")<br>
#@markdown > **del_source** -- нужно ли удалять исходный датасет<br>
#@markdown > **shuffle** -- нужно ли перемешивать данные перед разделением на части

#@markdown **RUN CELL: to split dataset on train/val/test.**

grammemer.data.load(suffix)
grammemer.data.split(
    *map(int, ratio.split('/')),
    del_source=del_source,
    shuffle=shuffle
    )

Набор train: 198 примеров.
Набор val: 24 примеров.
Набор test: 24 примеров.


---

In [9]:
#@title TRAIN NETWORK


epochs = 100 #@param {type: 'integer'}
batch_size = 24  #@param {type: 'integer'}
learning_rate = 0.001 #@param {type:"number"}
gamma = 0.99 #@param {type:"number"}
patience = 10 #@param {type: 'integer'}

#@markdown > **epochs** -- количество эпох обучения<br>
#@markdown > **batch_size** -- размер пакета<br>
#@markdown > **learning_rate** -- шаг обучения<br>
#@markdown > **gamma** -- коэффициент экспоненциального уменьшения шага обучения<br>
#@markdown > **patience** -- количество эпох без улучшения до уменьшения шага обучения на порядок

#@markdown **RUN CELL: to train network.**

parameters = {
    'epochs': epochs,
    'batch_size': batch_size,
    'learning_rate': learning_rate,
    'gamma': gamma,
    'patience': patience
}
grammemer.fit(**parameters)


     stage        learning_rate  epoch   batch_size  precision  recall   f1_score  loss  
198    training  369.73u        100.00  24.00          1.00    999.78m  999.89m    1.27m
199  validation  369.73u        100.00  24.00       788.70m    718.13m  751.76m   74.46m



---

In [10]:
#@title SAVE MODEL & HISTORY

suffix = '_demo'  #@param {type: 'string'}

#@markdown > **suffix** -- суффикс с которым будyт сохранёны файлы натренированной сети и истории обучения<br>

#@markdown **RUN CELL: to save model and learning history.**

grammemer.save(suffix)

---

In [11]:
#@title EVALUATE

batch_size = 24 #@param {type: 'integer'}

#@markdown > **batch_size** -- размер пакета<br>
#@markdown > Оценка сети осуществляется на тестовых данных.

#@markdown **RUN CELL: to test.**

grammemer.evaluate(batch_size=batch_size)

Progress: [evaluate]################################################################

[100.0%]֍[0:0:0]

precision    787.23m
recall       690.30m
f1_score     735.59m
loss          79.32m
dtype: float64

---
---

## INFERENCE

In [12]:
#@title LOAD PRETRAINED MODEL

suffix = ''  #@param {type: 'string'}

#@markdown > **suffix** -- суффикс имён файлов модели и истории для загрузки<br>

#@markdown **RUN CELL: to save model and learning history.**

%load_ext google.colab.data_table

grammemer.load()
grammemer.history.plot(10)
grammemer.history.data

,stage,learning_rate,epoch,batch_size,precision,recall,f1_score,loss
0,training,1.00m,1.00,32.00,913.54m,872.37m,889.43m,12.60m
1,validation,1.00m,1.00,32.00,944.55m,929.07m,936.72m,7.88m
2,training,990.00u,2.00,32.00,948.41m,935.60m,941.93m,7.24m
3,validation,990.00u,2.00,32.00,951.23m,941.71m,946.42m,6.76m
4,training,980.10u,3.00,32.00,954.48m,943.48m,948.92m,6.44m
...,...,...,...,...,...,...,...,...
101,validation,9.90u,51.00,64.00,985.32m,982.28m,983.79m,2.28m
102,training,9.80u,52.00,64.00,990.15m,987.31m,988.73m,1.53m
103,validation,9.80u,52.00,64.00,985.33m,982.38m,983.85m,2.27m
104,training,10.00u,53.00,128.00,988.35m,985.32m,986.83m,1.82m


---

In [13]:
#@title FINAL INFERENCE

x = 'Крылышкуя золотописьмом тончайших жил, кузнечик в кузов пуза уложил прибрежных много трав и вер.'  #@param {type: 'string'}
path = None #@param {type: 'raw'}

#@markdown > **x** -- строка, тензор или путь к txt-файлу<br>
#@markdown > **path** -- путь до файла для сохранения результата (если None, выводится на печать)

grammemer(x)


 ** SENTENCE 1 **

КРЫЛЫШКУЯ ['несовершенный вид', 'деепричастие', 'настоящее время']
ЗОЛОТОПИСЬМОМ ['неодушевлённые', 'творительный падеж', 'средний род', 'единственное число', 'существительное']
ТОНЧАЙШИХ ['превосходная степень', 'множественное число', 'полное прилагательное']
ЖИЛ ['одушевлённые', 'мужской род', 'множественное число', 'существительное']
КУЗНЕЧИК ['одушевлённые', 'мужской род', 'единственное число', 'существительное']
В ['аббревиатура', 'неодушевлённые', 'предложный падеж', 'мужской род', 'множественное число', 'существительное']
КУЗОВ ['неодушевлённые', 'именительный падеж', 'мужской род', 'единственное число', 'существительное']
ПУЗА ['одушевлённые', 'неодушевлённые', 'винительный падеж', 'средний род', 'единственное число', 'существительное']
УЛОЖИЛ ['совершенный вид', 'мужской род', 'изъявительное наклонение', 'единственное число', 'глагол', 'прошедшее время', 'переходный']
ПРИБРЕЖНЫХ ['родительный падеж', 'множественное число', 'полное прилагательное']
МНОГО ['н

# EOF